In [1]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

In [2]:
super_powers = pd.read_csv('Data/superhero_powers - superhero_powers.csv')
super_powers.head(3)

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."


In [3]:
super_info = pd.read_csv('Data/superhero_info - superhero_info.csv')
super_info.head(3)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"


# Clean the files and combine them into one final DataFrame.

    This dataframe should have the following columns:
        Hero (Just the name of the Hero) 
        Publisher
        Gender
        Eye color
        Race
        Hair color
        Height (numeric)
        Skin color
        Alignment
        Weight (numeric)
        Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
            Agility
            Flight
            Superspeed
            etc.

Hint: There is a space in "100 kg" or "52.5 cm"

## seperate the publisher from the hero name column

In [4]:

super_info[['Hero Name','Publisher']] = super_info['Hero|Publisher'].str.split('|', expand=True)
super_info = super_info.drop('Hero|Publisher', axis=1)
super_info.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero Name,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics


## Seperate the measurements column into height and weight numerical columns

In [5]:
#select one entry of the Measurements column
measurements = super_info.loc[0,'Measurements']
print(type(measurements))
measurements

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [6]:
#replace the single quotes with double quotes for use in JSON
measurements = measurements.replace("'",'"')
measurements

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [7]:
## now we can use JSON.loads to change from string to dictionary
fixed_measurements = json.loads(measurements)
print(type(fixed_measurements))
fixed_measurements

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [8]:
#now we can apply it to the whole Measurements column
#replace all sinlge quotes
super_info['Measurements'] = super_info['Measurements'].str.replace("'",'"')
#use JSON.load to change entire column to dictionaries
super_info['Measurements'] = super_info['Measurements'].apply(json.loads)
super_info['Measurements'].head(3)

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
Name: Measurements, dtype: object

## Unpack measurement dictionaries into Height and Weight numerical columns

In [9]:
height_weight = super_info['Measurements'].apply(pd.Series)
height_weight

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [10]:
#let's move the units up to the column name
height_weight.rename(columns={'Height':'Height (cm)', 'Weight':'Weight (kg)'}, inplace=True)
height_weight

,Height (cm),Weight (kg)
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [11]:
#next, we will replace the units with empty space using for loops
#characters to replace
to_replace = [' cm',' kg']
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    height_weight['Height (cm)'] = height_weight['Height (cm)'].str.replace(char,'',regex=False)
    height_weight['Weight (kg)'] = height_weight['Weight (kg)'].str.replace(char,'',regex=False)
height_weight.info()
height_weight.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Height (cm)  463 non-null    object
 1   Weight (kg)  463 non-null    object
dtypes: object(2)
memory usage: 7.4+ KB


,Height (cm),Weight (kg)
0,203.0,441.0
1,191.0,65.0
2,185.0,90.0
3,203.0,441.0
4,193.0,122.0


In [12]:
#convert columns to numerical
height_weight['Height (cm)'] = height_weight['Height (cm)'].astype(float)
height_weight['Weight (kg)'] = height_weight['Weight (kg)'].astype(float)
height_weight.info()
height_weight.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Height (cm)  463 non-null    float64
 1   Weight (kg)  463 non-null    float64
dtypes: float64(2)
memory usage: 7.4 KB


,Height (cm),Weight (kg)
0,203.0,441.0
1,191.0,65.0
2,185.0,90.0
3,203.0,441.0
4,193.0,122.0


In [13]:
#concat the new measurement columns to the super_info dataframe
super_info = pd.concat((super_info, height_weight),axis=1)
#drop the old Measurements column
super_info = super_info.drop(columns=['Measurements'])
super_info.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero Name,Publisher,Height (cm),Weight (kg)
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0


## OHE the power sets

In [14]:
super_powers.head(3)

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."


In [15]:
#split the Powers string
super_powers['Powers_split'] = super_powers['Powers'].str.split(',')
super_powers

,hero_names,Powers,Powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring]
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre..."
...,...,...,...
662,Yellowjacket II,"Flight,Energy Blasts,Size Changing","[Flight, Energy Blasts, Size Changing]"
663,Ymir,"Cold Resistance,Durability,Longevity,Super Str...","[Cold Resistance, Durability, Longevity, Super..."
664,Yoda,"Agility,Stealth,Danger Sense,Marksmanship,Weap...","[Agility, Stealth, Danger Sense, Marksmanship,..."
665,Zatanna,"Cryokinesis,Telepathy,Magic,Fire Control,Proba...","[Cryokinesis, Telepathy, Magic, Fire Control, ..."


In [16]:
#explode the split powers
exploded_powers = super_powers.explode('Powers_split')
exploded_powers

,hero_names,Powers,Powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Agility
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Strength
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Stamina
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Speed
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Accelerated Healing
...,...,...,...
665,Zatanna,"Cryokinesis,Telepathy,Magic,Fire Control,Proba...",Weather Control
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Super Speed
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Intangibility
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Time Travel


In [17]:
#use the unique powers from the split to get the column names to create
cols_to_make = exploded_powers['Powers_split'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [18]:
#for loop to create the OHE columns
for col in cols_to_make:
    super_powers[col] = super_powers['Powers'].str.contains(col)
super_powers.head()

C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super_powers[col] = super_powers['Powers'].str.contains(col)
C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super_powers[col] = super_powers['Powers'].str.contains(col)
C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super_powers[col] = super_powers['Powers'].str.contains(col)
C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super_powers[col] = super_powers['Powers'].str.contains(col)
C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super_powers[col] = super_powers['Powers'].str.contains(col)
C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super_powers[col] = super_powers['Powers'].str.contains(col)
C:\Users\Wombatmaster\AppData\Local\Temp\ipykernel_14768\3503384476.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

,hero_names,Powers,Powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
#lastlty, drop the temperory Powers_split and old Powers columns
super_powers = super_powers.drop(columns=['Powers','Powers_split'])
super_powers.head(3)

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


## merge the two dataframes

In [20]:
super_heroes = pd.merge(super_info, super_powers, left_on='Hero Name', right_on='hero_names')

In [21]:
super_heroes.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero Name,Publisher,Height (cm),Weight (kg),...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False


# Use your combined DataFrame to answer the following questions.

Compare the average weight of super powers who have Super Speed to those who do not.

In [22]:
super_heroes.groupby('Super Speed')['Weight (kg)'].mean()

Super Speed
False    101.773585
True     129.404040
Name: Weight (kg), dtype: float64

What is the average height of heroes for each publisher?

In [25]:
super_heroes.groupby('Publisher')['Height (cm)'].mean().sort_values()

Publisher
George Lucas         159.600000
Shueisha             171.500000
Dark Horse Comics    176.909091
Unknown              178.000000
Team Epic TV         180.750000
Star Trek            181.500000
DC Comics            181.923913
Marvel Comics        191.546128
Image Comics         211.000000
Name: Height (cm), dtype: float64